In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 800)  # 800 = 400*2 (400 points, 2d)

    def forward(self, x):  # x are the states: [d_obs_x, d_obs_y, d_goal_x, d_goal_y]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=6, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=800, bias=True)
)


In [2]:
import os
import numpy as np
from tqdm import tqdm

data_root_path = '/home/yigit/Documents/projects/irl_sfm/data/demonstrations/sfm/continuous_poses_1/new/combined/'

num_traj = 0
x_list, y_list = [], []

for filename in tqdm(os.listdir(data_root_path)):
    if filename.endswith(".npy"):
        filepath = os.path.join(data_root_path, filename)
        config = filename.strip('.npy').split('_')  # format: <sx>_<gx>_<ox>_<oy>.npy
        sx, sy = float(config[0]), 0.0
        gx, gy = float(config[1]), 13.0
        ox, oy = float(config[2]), float(config[3])
        
        x_list.append([sx, sy, gx, gy, ox, oy])
        trajectory = np.load(filepath, allow_pickle=True, encoding='latin1')
        desired_len = 400
        indices = np.linspace(0, len(trajectory)-1, desired_len, dtype=int)
        y_list.append(trajectory[indices, -3:-1].tolist())

print(len(x_list))
print(len(y_list))

100%|██████████| 5267/5267 [00:15<00:00, 335.31it/s]

5266
5266


In [3]:
batch_size = 100
num_instances_train = len(x_list)

x_train = torch.Tensor(x_list)
y_train = torch.Tensor(y_list).view(num_instances_train, desired_len*2)
print(y_train.shape)

torch.Size([5266, 800])


In [4]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

epochs = 2000

for epoch in range(epochs):  # loop over the dataset <epochs> times

    train_ids = torch.randperm(num_instances_train)  # shuffle instances every epoch
    num_iters = int(num_instances_train/batch_size)
    
    running_loss = 0.0
    for i in range(num_iters):
        batch_train_ids = train_ids[i*batch_size:(i+1)*batch_size]
        
        # get the inputs; data is a list of [inputs, labels]
        inputs = x_train[batch_train_ids, :]
        labels = y_train[batch_train_ids, :]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50), end = "\r")
            running_loss = 0.0
            
    if (epoch+1) % 10 == 0:
        print()

print('Finished Training')

/home/yigit/.local/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


[10,    50] loss: 1.000
[20,    50] loss: 0.290
[30,    50] loss: 0.205
[40,    50] loss: 0.204
[50,    50] loss: 0.202
[60,    50] loss: 0.198
[70,    50] loss: 0.194
[80,    50] loss: 0.193
[90,    50] loss: 0.193
[100,    50] loss: 0.191
[110,    50] loss: 0.190
[120,    50] loss: 0.188
[130,    50] loss: 0.189
[140,    50] loss: 0.188
[150,    50] loss: 0.187
[160,    50] loss: 0.185
[170,    50] loss: 0.184
[180,    50] loss: 0.182
[190,    50] loss: 0.182
[200,    50] loss: 0.182
[210,    50] loss: 0.181
[220,    50] loss: 0.180
[230,    50] loss: 0.180
[240,    50] loss: 0.179
[250,    50] loss: 0.178
[260,    50] loss: 0.177
[270,    50] loss: 0.175
[280,    50] loss: 0.175
[290,    50] loss: 0.175
[300,    50] loss: 0.173
[310,    50] loss: 0.170
[320,    50] loss: 0.171
[330,    50] loss: 0.169
[340,    50] loss: 0.168
[350,    50] loss: 0.166
[360,    50] loss: 0.165
[370,    50] loss: 0.164
[380,    50] loss: 0.165
[390,    50] loss: 0.160
[400,    50] loss: 0.160
[410,    

In [5]:
# Saving the trained model
model_root_path = f'/home/yigit/Documents/projects/irl_sfm/python_ws/nn_baseline_for_cnmp/model/nn_models/'
torch.save(net.state_dict(), f'{model_root_path}model_traj_est_state_dict_new_combined.pt')